# **ĐANG SUY NGHĨ XEM NÊN VIẾT CÁI GÌ ĐỂ GIỚI THIỆU**

# **Mount to Gdrive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Import requirement lib**

In [ ]:
import os # used to handle files using system commands.
import pickle # used to store numpy features extracted
import numpy as np  # used to perform a wide variety of mathematical operations 
                    # on arrays
from tqdm.notebook import tqdm  # progress bar decorator for iterators. 
                                # Includes a default range iterator printing to 
                                # stderr.

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input 
    # imported modules for feature extraction from the image data
from tensorflow.keras.preprocessing.image import load_img, img_to_array
    # used for loading the image and converting the image to a numpy array
from tensorflow.keras.preprocessing.text import Tokenizer
    # used for loading the text as convert them into a token
from tensorflow.keras.preprocessing.sequence import pad_sequences
    # used for equal distribution of words in sentences filling the remaining 
    # spaces with zeros
from tensorflow.keras.models import Model
    # stored model
from tensorflow.keras.utils import to_categorical, plot_model
    # used to visualize the architecture of the model through different images
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
    # operation for advanced model option

# **Apply API key to use kaggle API cmd**

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp '/content/gdrive/MyDrive/API key/kaggle.json' ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

## **Search && download dataset on kaggle**

In [ ]:
! kaggle datasets list --user 'adityajn105'

ref                     title                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------  ----------------------------  ----  -------------------  -------------  ---------  ---------------  
adityajn105/flickr8k    Flickr 8k Dataset              1GB  2020-04-27 07:27:19          28452        208  0.75             
adityajn105/flickr30k   Flick 30k Dataset              8GB  2020-04-27 08:07:18           1042          8  0.625            
adityajn105/glove6b50d  GLOVE 6B 50D Word Embeddings  68MB  2020-04-27 08:58:27            418          2  0.625            


In [ ]:
! kaggle datasets files 'adityajn105/flickr8k'

name          size  creationDate         
------------  ----  -------------------  
captions.txt   3MB  2020-04-27 07:27:19  


In [ ]:
! kaggle datasets download 'adityajn105/flickr8k' --unzip

 99% 1.03G/1.04G [00:09<00:00, 154MB/s]
100% 1.04G/1.04G [00:09<00:00, 118MB/s]


In [ ]:
! mkdir '/content/flickr8k'

In [ ]:
! mv '/content/captions.txt' '/content/flickr8k'

In [ ]:
! mv '/content/Images' '/content/flickr8k'

# **Determind DIR**

In [ ]:
BASE_DIR = '/content/flickr8k'

# **Extract Image Features**

## **Load VGG16 model**

In [ ]:
# load vgg16 model
model = VGG16()
# summarize
print(model.summary())

553467096/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

## **Restructure model**

Bỏ đi phần predict 1000 class vì không cần thiết.

Chỉ lấy đến phần trích xuất các đặc trưng ảnh.

In [ ]:
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summarize
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## **Extract feature**

In [ ]:
# extract features from image
features = {}
directory = os.path.join(BASE_DIR, 'Images')

for img_name in tqdm(os.listdir(directory)):
    # load the image from file
    img_path = directory + '/' + img_name
    image = load_img(img_path, target_size=(224, 224)) # default input size VGG16 (224, 224, 3)
    # convert image pixels to numpy array
    image = img_to_array(image)
    # reshape data for model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # preprocess image for vgg
    image = preprocess_input(image)
    # extract features
    feature = model.predict(image, verbose=0)
    # get image ID
    image_id = img_name.split('.')[0]
    # store feature
    features[image_id] = feature

  0%|          | 0/8091 [00:00<?, ?it/s]

## **Stored feature extracted**

In [ ]:
!mkdir '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature'

### **Stored as .pkl file**

In [ ]:
import pickle

# Save
with open('featured_extracted_flickr.pkl', 'wb') as f:
    pickle.dump(features, f)
        
# Load
with open('featured_extracted_flickr.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

print(type(loaded_dict), len(loaded_dict))

<class 'dict'> 8091


In [ ]:
! cp '/content/featured_extracted_flickr.pkl' '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature'

### **Stored as .npy file**

In [ ]:
import numpy as np

# Save
np.save('featured_extracted_flickr.npy', features) 

# Load
loaded_dict = np.load('featured_extracted_flickr.npy',allow_pickle='TRUE').item()

print(type(loaded_dict), len(loaded_dict))

<class 'dict'> 8091


In [ ]:
! cp '/content/featured_extracted_flickr.npy' '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature'

### **Make backup/recovery file**

In [ ]:
! mkdir '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature/Backup files'

In [ ]:
! cp '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature/featured_extracted_flickr.npy' '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature/Backup files'

In [ ]:
! cp '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature/featured_extracted_flickr.pkl' '/content/gdrive/MyDrive/Đồ án/Đồ án Thị giác máy tính nâng cao/Final/Extracted feature/Backup files'